In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/salary-data-with-age-and-experience/Salary_Data.csv
/kaggle/input/d/muazkl/alzheimer/alzheimer_gs.csv


In [ ]:
df = pd.read_csv("/kaggle/input/d/muazkl/alzheimer/alzheimer_gs.csv")

df = df.drop(['ds', 'Unnamed: 0'], axis=1)
# data = df.iloc[:,:-2]
# data = data.drop('Unnamed: 0', axis=1)
df2 = df.drop(['label'], axis=1)

label = df['label']
# experiment_label = df['ds']

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, label, test_size=0.2, random_state=42)
X_test = X_test.drop(['label'], axis=1)

In [ ]:
positive_samples = X_train[label == 1]
negative_samples = X_train[label == 0]

positive_count = positive_samples.shape[0]
negative_count = negative_samples.shape[0]

print("Positive sayısı:", positive_count)
print("Negative sayısı:", negative_count)

In [ ]:
negative_samples_resampled = negative_samples.sample(n=positive_count, replace=True, random_state=42)
positive_samples_resampled = positive_samples.sample(n=negative_count, replace=True, random_state=42)

balanced_negatives = pd.concat([positive_samples, negative_samples_resampled])
balanced_positives = pd.concat([negative_samples, positive_samples_resampled])

balanced_negatives_data = balanced_negatives.iloc[:, :-1]
balanced_negatives_label = balanced_negatives['label']

balanced_positives_data = balanced_positives.iloc[:, :-1]
balanced_positives_label = balanced_positives['label']

In [ ]:
from sklearn.model_selection import train_test_split

X_train_negatives = balanced_negatives_data
y_train_negatives = balanced_negatives_label

X_train_positives = balanced_positives_data
y_train_positives = balanced_positives_label

In [10]:
import tensorflow as tf
from tensorflow.keras import layers

In [11]:
def encoder():
  encoder_input = tf.keras.Input(shape=(9881,))
  encoder_layer = layers.Dense(2000, activation='relu')(encoder_input)
  encoder_output = layers.Dense(100, activation='relu')(encoder_layer)

  return tf.keras.Model(encoder_input, encoder_output)

def decoder(encoder_output):
  decoder_input = layers.Dense(100, activation='relu')(encoder_output)
  decoder_layer = layers.Dense(2000, activation='relu')(decoder_input)
  decoder_output = layers.Dense(9881, activation='relu')(decoder_layer)

  return tf.keras.Model(decoder_input, decoder_output)

def ae(encoder, decoder):

  return tf.keras.Model(encoder.input, decoder(encoder.output))

In [ ]:
encoder_negatives = encoder()
encoder_positives = encoder()
encoder_test = encoder()

In [ ]:
decoder_negatives = decoder(encoder_negatives.output)
decoder_positives = decoder(encoder_positives.output)
decoder_test = decoder(encoder_test.output)

In [ ]:
ae_negative_train = ae(encoder_negatives, decoder_negatives)

ae_positive_train = ae(encoder_positives, decoder_positives)

ae_test = ae(encoder_test, decoder_test)

In [ ]:
ae_negative_train.compile(optimizer='adam', loss='mse')
ae_negative_train.fit(X_train_negatives, X_train_negatives, epochs=200)

ae_positive_train.compile(optimizer='adam', loss='mse')
ae_positive_train.fit(X_train_positives, X_train_positives, epochs=200)

In [ ]:
ae_test.compile(optimizer='adam', loss='mse')
ae_test.fit(X_test, X_test, epochs=200)

In [ ]:
embedding_negative_train = encoder_negatives.predict(X_train_negatives)

embedding_positive_train = encoder_positives.predict(X_train_positives)

embedding_test = encoder_test.predict(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model_negatives = RandomForestClassifier(n_estimators=100)
rf_model_positives = RandomForestClassifier(n_estimators=100)

In [ ]:
rf_model_negatives.fit(embedding_negative_train, y_train_negatives)
rf_model_positives.fit(embedding_positive_train, y_train_positives)

In [ ]:
negative_preds = rf_model_negatives.predict(embedding_test)
positive_preds = rf_model_positives.predict(embedding_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_negatives = accuracy_score(y_test, negative_preds)
accuracy_positives = accuracy_score(y_test, positive_preds)

print(f'Accuracy for negative balanced samplings: {accuracy_negatives}' )
print(f'Accuracy for positive balanced samplings: {accuracy_positives}' )

In [5]:
data_test = df.iloc[:, :-1]
label_test = df['label']

df_filtered = df[df['label']==1]

data_train = df_filtered.iloc[:, :-1]
label_train = df_filtered['label']

X_train, X_test, y_train, y_test = train_test_split(data_test, label_test, test_size=0.5, random_state=42)

In [12]:
encoder = encoder()
decoder = decoder(encoder.output)
ae = ae(encoder, decoder)

In [ ]:
data_train

In [ ]:
ae.compile(optimizer='adam', loss='mse')
ae.fit(data_train, data_train, epochs=200)
ae.trainable = False

In [ ]:
samples_count = X_test.shape[0]
classificated_labels = []

for i in range(samples_count):
    sample_x = X_test.iloc[i]
    sample_x = sample_x.to_frame().T

    fitted = ae.fit(sample_x, sample_x, epochs=1)
    loss = fitted.history['loss'][0]
    print(list(y_test)[i])
  
    if loss>0.75:
        classificated_labels.append(0)
    else:
        classificated_labels.append(1)

Fit etmeden önce model freeze edilmelidir.

In [29]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, classificated_labels)
print(accuracy)

0.4308300395256917
